

# to_onnx and a custom operator inplace

This example shows how to convert a custom operator as defined
in the tutorial [Python Custom Operators](https://pytorch.org/tutorials/advanced/python_custom_ops.html#python-custom-ops-tutorial).

Inplace modification are not supported by onnx.

## A model with a custom ops


In [ ]:
from typing import Any, Dict, List
import numpy as np
import torch
from onnx_array_api.plotting.graphviz_helper import plot_dot
from experimental_experiment.xbuilder import GraphBuilder
from experimental_experiment.helpers import pretty_onnx
from experimental_experiment.torch_interpreter import to_onnx, Dispatcher, ExportOptions

We define a model with a custom operator.



In [ ]:
@torch.library.custom_op("mylib::numpy_sin", mutates_args={"output"}, device_types="cpu")
def numpy_sin(x: torch.Tensor, output: torch.Tensor) -> None:
    assert x.device == output.device
    assert x.device.type == "cpu"
    x_np = x.numpy()
    output_np = output.numpy()
    np.sin(x_np, out=output_np)


class ModuleWithACustomOperator(torch.nn.Module):
    def forward(self, x):
        out = torch.zeros(x.shape)
        numpy_sin(x, out)
        return out


model = ModuleWithACustomOperator()

Let's check it runs.



In [ ]:
x = torch.randn(1, 3)
model(x)

As expected, it does not export.



In [ ]:
try:
    torch.export.export(model, (x,))
    raise AssertionError("This export should failed unless pytorch now supports this model.")
except Exception as e:
    print(e)

The exporter fails with the same eror as it expects torch.export.export to work.



In [ ]:
try:
    to_onnx(model, (x,))
except Exception as e:
    print(e)

## Registration

The exporter how to convert the new exporter into ONNX.
This must be defined. The first piece is to tell the exporter
that the shape of the output is the same as x.
input names must be the same.



In [ ]:
@numpy_sin.register_fake
def numpy_sin_shape(x, output):
    pass

Let's see what the fx graph looks like.



In [ ]:
print(torch.export.export(model, (x,)).graph)

Next is the conversion to onnx.



In [ ]:
T = str  # a tensor name


def numpy_sin_to_onnx(
    g: GraphBuilder,
    sts: Dict[str, Any],
    outputs: List[str],
    x: T,
    name: str = "mylib.numpy_sin",
    **_kwargs,
) -> T:
    # name= ... lets the user know when the node comes from
    # o is not used, we could check the shape are equal.
    # outputs contains unexpectedly two outputs
    g.op.Sin(x, name=name, outputs=outputs[1:])
    return outputs

We create a :class:`Dispatcher <experimental_experiment.torch_interpreter.Dispatcher>`.



In [ ]:
dispatcher = Dispatcher({"mylib::numpy_sin": numpy_sin_to_onnx})

And we convert again.



In [ ]:
onx = to_onnx(
    model,
    (x,),
    dispatcher=dispatcher,
    optimize=False,
    export_options=ExportOptions(decomposition_table="default"),
)
print(pretty_onnx(onx))

And we convert again with optimization this time.



In [ ]:
onx = to_onnx(
    model,
    (x,),
    dispatcher=dispatcher,
    optimize=True,
    export_options=ExportOptions(decomposition_table="default"),
)
print(pretty_onnx(onx))

And visually.



In [ ]:
plot_dot(onx)